# Import dependencies and environment variables

In [6]:
import os
import time
from dotenv import load_dotenv

import yt_dlp
from moviepy.editor import VideoFileClip
import whisper
from pyannote.audio import Pipeline
import torch
import ffmpeg

from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document

from langchain.chains import (
    create_retrieval_chain,
    RetrievalQA,
)

from langgraph.checkpoint import MemorySaver
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage

from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import tool

from langgraph.prebuilt import create_react_agent

import ipywidgets as widgets
from IPython.display import display

from langchain.agents import Tool
from langsmith.evaluation import evaluate
from langchain import hub
from langsmith import Client
from langchain.schema import HumanMessage
import json

In [7]:
# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
ELEVEN_API_KEY = os.getenv('ELEVEN_API_KEY')
HF_TOKEN = os.getenv('HF_TOKEN')

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY environment variable not set")
elif not LANGCHAIN_API_KEY:
    raise ValueError("LANGCHAIN_API_KEY environment variable not set")

In [10]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Ironhack_Project3"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

# Set up directories
os.makedirs('uploads', exist_ok=True)
os.makedirs('downloads', exist_ok=True)

# Video-processing functions

In [9]:
# Function to handle file uploads
def handle_file_upload():
    uploader = widgets.FileUpload(accept='video/*', multiple=False)
    display(uploader)
    return uploader

# Function to save uploaded file
def save_uploaded_file(file):
    start_time = time.time()
    if file.value:
        for file_info in file.value:
            filename = file_info['name']
            content = file_info['content']
            print(f"Filename: {filename}")
            file_path = os.path.join('uploads', filename)
            with open(file_path, 'wb') as f:
                f.write(content)
            end_time = time.time()
            duration = end_time - start_time
            print(f"File {filename} uploaded successfully to {file_path} in {duration:.2f} seconds")
            return file_path, filename, duration
    else:
        print("No file uploaded.")
        return None, None, 0

# Function to extract metadata using ffmpeg
def extract_metadata_ffmpeg(video_path):
    try:
        probe = ffmpeg.probe(video_path)
        video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
        metadata = {
            "duration": float(video_info['duration']),
            "width": int(video_info['width']),
            "height": int(video_info['height']),
            "codec_name": video_info['codec_name'],
            "bit_rate": int(video_info['bit_rate'])
        }
        print(f"Extracted metadata: {metadata}")
        return metadata
    except Exception as e:
        print(f"Error extracting metadata with ffmpeg: {e}")
        return None

# Function to download video from YouTube
def download_youtube_video(url):
    start_time = time.time()
    ydl_opts = {
        'format': 'mp4',
        'outtmpl': 'downloads/video.%(ext)s',
        'verbose': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            
            metadata = {
                "title": info_dict.get('title', 'video'),
                "id": info_dict.get('id'),
                "duration": info_dict.get('duration'),
                "upload_date": info_dict.get('upload_date'),
                "uploader": info_dict.get('uploader'),
                "uploader_id": info_dict.get('uploader_id'),
                "view_count": info_dict.get('view_count'),
                "like_count": info_dict.get('like_count'),
                "dislike_count": info_dict.get('dislike_count'),
                "average_rating": info_dict.get('average_rating'),
                "age_limit": info_dict.get('age_limit'),
                "categories": ", ".join(info_dict.get('categories', [])),
                "tags": ", ".join(info_dict.get('tags', [])),
                "ext": info_dict.get('ext'),
                "thumbnail": info_dict.get('thumbnail'),
                "description": info_dict.get('description'),
                "channel": info_dict.get('channel'),
                "channel_id": info_dict.get('channel_id'),
                "is_live": info_dict.get('is_live'),
                "release_date": info_dict.get('release_date'),
                "availability": info_dict.get('availability')
            }
            
            for key, value in metadata.items():
                if value is None:
                    metadata[key] = "Empty"
            
            print(f"Video title: {metadata['title']}")

            video_ext = metadata['ext']
            initial_path = os.path.abspath(f'downloads/video.{video_ext}')
            if not os.path.isfile(initial_path):
                raise FileNotFoundError(f"Downloaded video file not found: {initial_path}")

            counter = 1
            final_path = os.path.abspath(f'downloads/video_{counter}.{video_ext}')
            while os.path.isfile(final_path):
                counter += 1
                final_path = os.path.abspath(f'downloads/video_{counter}.{video_ext}')

            os.rename(initial_path, final_path)
            print(f"Downloaded video saved to: {final_path}")

            end_time = time.time()
            duration = end_time - start_time
            print(f"Time taken for downloading video: {duration:.2f} seconds")
            
            return final_path, metadata, duration
    except Exception as e:
        print(f"Error downloading video: {e}")
        return None, None, 0

# Function to extract audio from video
def extract_audio(video_path):
    start_time = time.time()
    try:
        video = VideoFileClip(video_path)
        audio_path = video_path.replace('.mp4', '.wav')
        audio_path = os.path.abspath(audio_path)
        print(f"Extracting audio to: {audio_path}")
        video.audio.write_audiofile(audio_path)
        if not os.path.isfile(audio_path):
            raise FileNotFoundError(f"Extracted audio file not found: {audio_path}")
        end_time = time.time()
        duration = end_time - start_time
        print(f"Audio extracted in {duration:.2f} seconds")
        return audio_path, duration
    except Exception as e:
        print(f"Error extracting audio: {e}")
        return None, 0

# Function to select Whisper model based on duration and mode
def select_whisper_model(duration, mode='Fast'):
    if mode == "Accurate":
        if duration > 3600:  # More than 1 hour
            model_name = "tiny"
        elif duration > 1800:  # More than 30 minutes
            model_name = "base"
        elif duration > 600:  # More than 10 minutes
            model_name = "small"
        else:  # 10 minutes or less
            model_name = "medium"
    else:  # Fast mode
        if duration > 1800:  # More than 30 minutes
            model_name = "tiny"
        elif duration > 600:  # More than 10 minutes
            model_name = "base"
        else:  # 10 minutes or less
            model_name = "small"
    
    print(f"Selected Whisper model: {model_name}")
    return model_name

# Function to transcribe audio
def transcribe_audio(audio_path, duration, mode='Fast'):
    start_time = time.time()
    try:
        print(f"Transcribing audio from: {audio_path}")
        if not os.path.isfile(audio_path):
            raise FileNotFoundError(f"Audio file not found: {audio_path}")

        model_name = select_whisper_model(duration, mode)
        model = whisper.load_model(model_name)
        
        # Check if CUDA is available and use it
        if torch.cuda.is_available():
            model = model.to("cuda")
            print("Using CUDA for Whisper transcription")
        
        result = model.transcribe(audio_path)
        end_time = time.time()
        transcription_duration = end_time - start_time
        print(f"Transcription completed in {transcription_duration:.2f} seconds.")
        return result['text'], transcription_duration
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return "", 0

# Function to transcribe audio with timestamps
def transcribe_audio_with_timestamps(audio_path, duration, mode='Fast'):
    start_time = time.time()
    try:
        print(f"Transcribing audio from: {audio_path}")
        if not os.path.isfile(audio_path):
            raise FileNotFoundError(f"Audio file not found: {audio_path}")

        model_name = select_whisper_model(duration, mode)
        model = whisper.load_model(model_name)
        
        # Check if CUDA is available and use it
        if torch.cuda.is_available():
            model = model.to("cuda")
            print("Using CUDA for Whisper transcription")
        
        result = model.transcribe(audio_path, word_timestamps=True)
        end_time = time.time()
        transcription_duration = end_time - start_time
        print(f"Transcription with timestamps completed in {transcription_duration:.2f} seconds.")
        return result, transcription_duration
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return {}, 0

# Function to combine metadata and transcription
def combine_metadata_and_transcription(metadata, transcription):
    combined_text = "Metadata:\n"
    for key, value in metadata.items():
        combined_text += f"{key}: {value}\n"
    combined_text += "\nTranscription:\n" + transcription
    return combined_text

# Function to combine metadata, transcription, and diarization
def combine_metadata_transcription_diarization(metadata, transcription, diarization):
    combined_text = "Metadata:\n"
    for key, value in metadata.items():
        combined_text += f"{key}: {value}\n"
    combined_text += "\nDiarization:\n"
    for segment in diarization.itertracks(yield_label=True):
        speaker = segment[2]
        start_time = segment[0].start
        end_time = segment[0].end
        combined_text += f"Speaker {speaker} [{start_time:.2f} - {end_time:.2f}]: {transcription['segments'][0]['text']}\n"
    return combined_text

# Function to perform diarization
def perform_diarization(audio_path, duration, mode='Fast'):
    start_time = time.time()
    try:
        pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=os.getenv("HF_TOKEN"))

        # Check if CUDA is available and use it
        if torch.cuda.is_available():
            pipeline.to(torch.device("cuda"))
            print("Using CUDA for Pyannote diarization")
        
        diarization_result = pipeline(audio_path)
        end_time = time.time()
        diarization_duration = end_time - start_time
        print(f"Diarization completed in {diarization_duration:.2f} seconds.")
        return diarization_result, diarization_duration
    except Exception as e:
        print(f"Error during diarization: {e}")
        return None, 0

# Main function to process video
def process_video(source_type, source, mode='Fast', process_type='Transcription'):
    timings = {}
    total_start_time = time.time()
    
    if source_type == 'upload':
        video_path, filename, upload_duration = save_uploaded_file(source)
        metadata = extract_metadata_ffmpeg(video_path)
        timings['upload'] = upload_duration
    elif source_type == 'youtube':
        video_path, metadata, download_duration = download_youtube_video(source)
        timings['download'] = download_duration
    else:
        print("Invalid source type.")
        return None
    
    if not video_path or not metadata:
        print("Failed to get video or metadata.")
        return None
    
    audio_path, extract_duration = extract_audio(video_path)
    if not audio_path:
        print("Failed to extract audio.")
        return None
    timings['extract_audio'] = extract_duration
    
    if process_type == 'Transcription':
        transcription, transcribe_duration = transcribe_audio(audio_path, metadata['duration'], mode)
        if not transcription:
            print("Failed to transcribe audio.")
            return None
        timings['transcribe_audio'] = transcribe_duration
        combined_text = combine_metadata_and_transcription(metadata, transcription)
    elif process_type == 'Diarization':
        transcription_result, transcribe_duration = transcribe_audio_with_timestamps(audio_path, metadata['duration'], mode)
        if not transcription_result:
            print("Failed to transcribe audio with timestamps.")
            return None
        timings['transcribe_audio_with_timestamps'] = transcribe_duration
        
        diarization_result, diarization_duration = perform_diarization(audio_path, metadata['duration'], mode)
        if not diarization_result:
            print("Failed to perform diarization.")
            return None
        timings['diarization'] = diarization_duration
        
        combined_text = combine_metadata_transcription_diarization(metadata, transcription_result, diarization_result)
    
    total_end_time = time.time()
    timings['total_processing'] = total_end_time - total_start_time
    
    print(combined_text)
    print("Timings:", timings)
    
    document = Document(page_content=combined_text, metadata=metadata)
    
    # Clean up files
    try:
        os.remove(video_path)
        os.remove(audio_path)
    except Exception as e:
        print(f"Error deleting files: {e}")
    
    return document

# Function to create vectorstore
def create_vectorstore(document):
    try:
        vectorstore = Chroma.from_documents(documents=[document], embedding=OpenAIEmbeddings())
        retriever = vectorstore.as_retriever()
        return retriever
    except Exception as e:
        print(f"Error creating vectorstore: {e}")
        return None


retriever = None

def on_process_button_clicked(b):
    global retriever
    if url_input.value:
        source_type = 'youtube'
        source = url_input.value
    elif uploaded_file_widget.value:
        source_type = 'upload'
        source = uploaded_file_widget
    else:
        print("Please provide a YouTube URL or upload a file.")
        return
    
    document = process_video(source_type, source, mode_selector.value, process_type_selector.value)
    if document:
        try:
            vectorstore = Chroma.from_documents(documents=[document], embedding=OpenAIEmbeddings())
            retriever = vectorstore.as_retriever()
        except Exception as e:
            print(f"Error creating vectorstore: {e}")
            exit()
    
        if retriever:
            print("Vectorstore created and retriever initialized.")
        else:
            print("Failed to create vectorstore.")
    else:
        print("Failed to process video.")


# Chatbot

In [5]:
# Set up the chatbot class
class VideoChatbot:
    def __init__(self):
        self.retriever = None
        self.qa_chain = None
        self.memory = MemorySaver()
        self.model = ChatOpenAI(model="gpt-4o", temperature=0)
        self.prompt = '''You are a chatbot that answers questions and performs tasks about a video that the user provides. 
                         Never ask the user to provide a video without first checking if there is one already.
                         If lacking context, assume the user is always talking about the video.
                         First, consider which tools you need to use, if any.
                         When retrieving information, consider that the transcription might not be perfect every time.
                         Then, if relevant, try to identify speakers by their names or usernames, using their dialogue and considering the available metadata.
                         Then use more steps when needed in order to get the right answer. 
                         Finally, you must always identify the language the user is utilizing in their last message and answer in that language, unless the user tells you otherwise.
                      '''
        self.agent = None

    def initialize_qa_chain(self):
        llm = ChatOpenAI(model="gpt-4o")
        try:
            qa = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=self.retriever
            )
            print("QA chain initialized successfully.")
            self.qa_chain = qa
        except Exception as e:
            print(f"Error initializing QA chains: {e}")
            self.qa_chain = None

    def create_agent(self):
        tools = [
            Tool(
                name='video_transcript_retriever',
                func=self.qa_chain.run,
                description=(
                    'Searches and returns excerpts from the transcript of the user uploaded video.'
                )
            ),
        ]
        self.agent = create_react_agent(self.model, tools=tools, messages_modifier=self.prompt, checkpointer=self.memory)
        print("Agent created successfully.")

    def process_query(self, query):
        if not self.agent:
            print("Agent not initialized.")
            return

        inputs = {"messages": [("user", query)]}
        config = {"configurable": {"thread_id": "2"}}
        stream = self.agent.stream(inputs, config=config, stream_mode="values")
        for s in stream:
            message = s["messages"][-1]
            if isinstance(message, tuple):
                print(message)
            else:
                message.pretty_print()

# Create instance of the chatbot
chatbot = VideoChatbot()

# Example usage of the chatbot after processing the video
def on_process_button_clicked(b):
    if url_input.value:
        source_type = 'youtube'
        source = url_input.value
    elif uploaded_file_widget.value:
        source_type = 'upload'
        source = uploaded_file_widget
    else:
        print("Please provide a YouTube URL or upload a file.")
        return
    
    document = process_video(source_type, source, mode_selector.value, process_type_selector.value)
    if document:
        try:
            vectorstore = Chroma.from_documents(documents=[document], embedding=OpenAIEmbeddings())
            chatbot.retriever = vectorstore.as_retriever()
        except Exception as e:
            print(f"Error creating vectorstore: {e}")
            return
    
        if chatbot.retriever:
            print("Vectorstore created and retriever initialized.")
            chatbot.initialize_qa_chain()
            chatbot.create_agent()
            example_query()  # Run example query after agent creation
        else:
            print("Failed to create vectorstore.")
    else:
        print("Failed to process video.")

# Display widgets and process button
uploaded_file_widget = handle_file_upload()
url_input = widgets.Text(description="YouTube URL:")
mode_selector = widgets.Dropdown(options=["Fast", "Accurate"], description="Mode:")
process_type_selector = widgets.Dropdown(options=["Transcription", "Diarization"], description="Process Type:")
process_button = widgets.Button(description="Process Video")
process_button.on_click(on_process_button_clicked)
display(url_input, uploaded_file_widget, mode_selector, process_type_selector, process_button)

# Example query to the chatbot
def example_query():
    if chatbot.agent:
        query = "What is the video about?"
        chatbot.process_query(query)
    else:
        print("Agent not initialized.")


FileUpload(value=(), accept='video/*', description='Upload')

Text(value='', description='YouTube URL:')

FileUpload(value=(), accept='video/*', description='Upload')

Dropdown(description='Mode:', options=('Fast', 'Accurate'), value='Fast')

Dropdown(description='Process Type:', options=('Transcription', 'Diarization'), value='Transcription')

Button(description='Process Video', style=ButtonStyle())

In [9]:
query = "What is the healer's punchline?"
chatbot.process_query(query)

================================ Human Message =================================

What is the healer's punchline?
================================== Ai Message ==================================
Tool Calls:
  video_transcript_retriever (call_RCB5pkuIRLGe81VrcH9WLaCI)
 Call ID: call_RCB5pkuIRLGe81VrcH9WLaCI
  Args:
    __arg1: healer punchline


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


================================= Tool Message =================================
Name: video_transcript_retriever

The punchline of the video "When the healer thinks they're DPS" comes at the end when, despite being repeatedly told to focus on healing, Adam (the healer) starts the fight with a battle cry of "Holy daga!" and immediately dies. This leads the rest of the group to conclude, "He's dead. He's already dead. He's dead. Should we go in? No. We don't have a healer. Let's go find another healer. Goodbye." This highlights the comedic frustration of the group trying to get their healer to do their assigned role.
================================== Ai Message ==================================

The healer's punchline in the video is when Adam, despite being told to focus on healing, starts the fight with a battle cry of "Holy daga!" and immediately dies. This leads the group to humorously conclude, "He's dead. He's already dead. He's dead. Should we go in? No. We don't have a healer.

# Evaluation

We are going to build 2 evaluation sets: for Q&A and for summarization, 2 of the main tasks the model is going to execute. They have to be oriented for RAG

In order to get them right, we will use LangSmith https://docs.smith.langchain.com/tutorials/Developers/rag#evaluating-intermediate-steps and DeepEval https://docs.confident-ai.com/docs/guides-rag-evaluation

We first need to create our datasets

In [ ]:
config = {"configurable": {"thread_id": "3"}}

In [ ]:
### Dataset name
dataset_name = "Video_Test"

client = Client()

# Define dataset: these are your test cases
dataset = client.create_dataset(dataset_name)

In [ ]:
client.create_examples(
    inputs=[
        {"input_question": "Hello"},
        {"input_question": "What is the video about?"},
        {"input_question": "What was the last miniature that was released for the Thousand Sons?"},
        {"input_question": "What is the most likely miniature to be released for Imperial Agents?"},
        {"input_question": "What are the Space Wolves units that are less in need of an update?"},
        {"input_question": "What can we expect the Black Templars to get if we are very optimistic?"},
        {"input_question": "What is a common practice from Games Workshop when releasing a new codex?"},
    ],
    outputs=[
        {"output_answer": "Hello, how can I help you?."},
        {"output_answer": "The video is about all the Warhammer 40K factions that are still missing a Codex in 10th edition and what novelties will the codexes bring when they are released, focusing on new possible miniatures."},
        {"output_answer": "The Infernal Master."},
        {"output_answer": "Inquisitor Coteaz."},
        {"output_answer": "Wulfen and Thunderwolves."},
        {"output_answer": "Some themed Terminators."},
        {"output_answer": "They release at least one miniature, usually one or two characters and often releasing Battleforces."},
    ],
    dataset_id=dataset.id,
)

In [ ]:
def extract_final_answer(messages):
    """
    Extracts the content of the last AIMessage from the messages.
    
    Args:
    messages (list): List of message dictionaries containing messages from human, AI, and tools.
    
    Returns:
    str: The content of the last AIMessage.
    """
    # Iterate over the messages in reverse order to find the last AIMessage
    for message in reversed(messages):
        # Check if the message is an instance of AIMessage
        if isinstance(message, AIMessage):
            return message.content
    return ''


def predict_rag_answer(example: dict):
    #Use this for answer evaluation
    query = example["input_question"]
    # Format inputs properly
    inputs = {"messages": [{"role": "user", "content": query}]}
    answer = agent.invoke(inputs, config=config, stream_mode="values")
    
    if 'messages' in answer:
        response = extract_final_answer(answer['messages'])
    else:
        response = "No valid response found."
    
    return {"answer": response}

In [ ]:
#Implement this if there is time, for evaluating correctly the retrieved documents and hallucinations.

"""
#Implement this if there is time, for evaluating correctly the retrieved documents and hallucinations.
def predict_rag_answer_with_context(example: dict):
    #Use this for answer evaluation
    query = example["input_question"]
    # Format inputs properly
    inputs = {"messages": [{"role": "user", "content": query}]}
    answer = agent.invoke(inputs, config=config, stream_mode="values")
    
    if 'messages' in answer:
        response = extract_final_answer(answer['messages'])
    else:
        response = "No valid response found."
    
    return {"answer": response}
"""

### Response vs reference answer

In [ ]:
# Grade prompt
grade_prompt_answer_accuracy = hub.pull("langchain-ai/rag-answer-vs-reference")

def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    # Access example correctly
    input_question = example.inputs["input_question"]
    reference = example.outputs["output_answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_answer_accuracy | llm

    # Run evaluator
    score = answer_grader.invoke({
        "question": input_question,
        "correct_answer": reference,
        "student_answer": prediction
    })
    score = score["Score"]

    return {"key": "answer_v_reference_score", "score": score}

### Response vs input

In [ ]:
# Grade prompt
grade_prompt_answer_helpfulness = prompt = hub.pull("langchain-ai/rag-answer-helpfulness")

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer helpfulness
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs["input_question"]
    prediction = run.outputs["answer"]

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_answer_helpfulness | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_helpfulness_score", "score": score}

### Response vs retrieved docs

In [ ]:
# Prompt
grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """

    # RAG inputs
    input_question = example.inputs["input_question"]
    contexts = run.outputs.get("contexts", [])

    # RAG answer
    prediction = run.outputs.get("answer", "No valid response found.")

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}


### Retrieved docs vs input

In [ ]:
# Grade prompt
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """

    # RAG inputs
    input_question = example.inputs["input_question"]
    contexts = run.outputs.get("contexts", [])

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

## Run evaluators

In [ ]:
from langsmith.evaluation import evaluate

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[
        answer_evaluator,
        answer_helpfulness_evaluator,
        answer_hallucination_evaluator,
        docs_relevance_evaluator
    ],
    experiment_prefix="Full_final_test",
    metadata={"version": "Video_final_test, ChatMistralAI"},
)

# Used package versions for requirements.txt

In [2]:
import pkg_resources

# List of packages you want to check
packages = [
    'Flask', 'yt-dlp', 'moviepy', 'whisper', 'pyannote.audio', 'torch',
    'ffmpeg-python', 'python-dotenv', 'langchain-openai', 'langchain',
    'langgraph', 'ipywidgets', 'IPython', 'langchain-community'
]

# Get the installed version of each package
installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set if pkg.key in [p.lower() for p in packages]}

# Print the package versions
for pkg, version in installed_packages.items():
    print(f"{pkg}=={version}")


ffmpeg-python==0.2.0
flask==3.0.3
ipython==8.15.0
ipywidgets==8.1.3
langchain==0.2.7
langchain-community==0.2.6
langchain-openai==0.1.16
langgraph==0.1.8
moviepy==1.0.3
pyannote.audio==3.3.1
python-dotenv==1.0.1
torch==2.3.1
whisper==1.1.10
yt-dlp==2024.7.9
